In [1]:
import time
import requests
import pandas as pd
import datetime

In [2]:
import hashlib
import hmac
import base64


class Signature:

    @staticmethod
    def generate(timestamp, method, uri, secret_key):
        message = "{}.{}.{}".format(timestamp, method, uri)
        hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)

        hash.hexdigest()
        return base64.b64encode(hash.digest())

In [3]:
BASE_URL = 'https://api.searchad.naver.com'
SECRET_KEY = '---'
API_KEY = '---'
CUSTOMER_ID = '---'

In [4]:
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

In [5]:
uri = '/ncc/campaigns'
method = 'GET'
r = requests.get(BASE_URL + uri, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

budget_list = r.json()

In [6]:
df_ratio = pd.read_csv("./budget_ratio.csv")

In [7]:
df = pd.read_csv("./budget.csv")
df['date'] = pd.to_datetime(df['date'])
today = pd.to_datetime(datetime.date.today())
budget_amount = df.query('date == @today').iloc[0, 1]

In [8]:
for i in df_ratio.index:
    for j in budget_list:
        if j['nccCampaignId'] == df_ratio['id'][i]:
            target_campaign = j
            target_campaign['dailyBudget'] = int(round(budget_amount * df_ratio['ratio'][i], -4))
            # print(target_campaign['dailyBudget'])

            uri = '/ncc/campaigns/' + target_campaign['nccCampaignId']
            method = 'PUT'
            r = requests.put(BASE_URL + uri, params = {'fields' : 'budget'}, json = target_campaign, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

            print("response status_code = {}".format(r.status_code))
            print("response body = {}".format(r.json()))

response status_code = 200
response body = {'nccCampaignId': 'cmp-a001-01-000000005946580', 'customerId': 985635, 'name': '1. 파워링크_렌터카_MO_중요', 'userLock': False, 'campaignTp': 'WEB_SITE', 'deliveryMethod': 'ACCELERATED', 'trackingMode': 'AUTO_TRACKING_MODE', 'usePeriod': False, 'dailyBudget': 2700000, 'useDailyBudget': True, 'totalChargeCost': 715022, 'status': 'ELIGIBLE', 'statusReason': 'ELIGIBLE', 'expectCost': 6050, 'migType': 0, 'delFlag': False, 'regTm': '2022-10-26T07:00:24.000Z', 'editTm': '2024-08-20T00:55:36.000Z'}
response status_code = 200
response body = {'nccCampaignId': 'cmp-a001-01-000000005942643', 'customerId': 985635, 'name': '1. 파워링크_렌터카_PC_중요', 'userLock': False, 'campaignTp': 'WEB_SITE', 'deliveryMethod': 'ACCELERATED', 'trackingMode': 'AUTO_TRACKING_MODE', 'usePeriod': False, 'dailyBudget': 1080000, 'useDailyBudget': True, 'totalChargeCost': 315491, 'status': 'ELIGIBLE', 'statusReason': 'ELIGIBLE', 'expectCost': 3762, 'migType': 0, 'delFlag': False, 'regTm': '202